<h1 align="center" style="color:#96d9f0;">Computational Intelligence for
Optimization - Project</h1>
<h3 align="center" style="color:#96d9f0;">Group AW - Wedding Seating Optimization</h3>

---

### <span style="color:#96d9f0;">Group Members</span>

<table>
  <thead style="color:#96d9f0;">
    <tr>
      <th>Name</th>
      <th>Email</th>
      <th>Student ID</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Afonso Dias</td>
      <td>20211540@novaims.unl.pt</td>
      <td>20211540</td>
    </tr>
    <tr>
      <td>Isabel Duarte</td>
      <td>20240545@novaims.unl.pt</td>
      <td>20240545</td>
    </tr>
    <tr>
      <td>Pedro Campino</td>
      <td>20240537@novaims.unl.pt</td>
      <td>20240537</td>
    </tr>
    <tr>
      <td>Rita Matos</td>
      <td>20211642@novaims.unl.pt</td>
      <td>20211642</td>
    </tr>
  </tbody>
</table>

In [1]:
#future improvements: test random fitnesses to set a baseline average to compare our results

In [2]:
#compare performance of different selection methods, crossovers, and mutations

---

### <span style="color:#96d9f0;">1. Imports</span>

This section includes all the necessary library imports required for the development of the project. We also use the library provided during the practical classes, as recommended, to support the implementation. Additionally, the relationship matrix used for evaluating guest compatibility is imported here.

In [3]:
import sys
sys.path.append('..')

In [4]:
import random
from copy import deepcopy
from library.solution import Solution
import csv
import os
import pandas as pd
import numpy as np
from itertools import product
from tqdm import tqdm 
from itertools import combinations
from typing import Callable
from library.algorithms.genetic_algorithms.algorithm import get_best_ind

In [5]:
# Import the relationship matrix as a pandas dataframe for visualization

relationship_matrix_df = pd.read_csv('data/seating_data.csv')
relationship_matrix_df.drop(columns='idx', inplace=True)
relationship_matrix_df

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
0,0,5000,0,0,700,700,0,0,0,0,...,100,100,0,0,100,100,100,0,0,0
1,5000,0,700,700,0,0,300,300,500,500,...,100,100,0,100,0,0,0,0,0,0
2,0,700,0,2000,0,0,0,0,300,300,...,0,0,0,0,0,0,0,0,0,0
3,0,700,2000,0,0,0,900,400,300,300,...,0,0,0,0,0,0,0,0,0,0
4,700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60,100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,100,0,0,2000,700,700
61,0,0,0,0,0,0,0,0,0,0,...,0,0,-1000,0,100,0,2000,0,700,700
62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,700,700,0,900


In [6]:
# Import the relationship matrix as a numpy array

relationship_matrix = np.loadtxt('data/seating_data.csv', delimiter=',', skiprows=1)
relationship_matrix = relationship_matrix[ : , 1:]
relationship_matrix 

array([[   0., 5000.,    0., ...,    0.,    0.,    0.],
       [5000.,    0.,  700., ...,    0.,    0.,    0.],
       [   0.,  700.,    0., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ...,    0.,  700.,  700.],
       [   0.,    0.,    0., ...,  700.,    0.,  900.],
       [   0.,    0.,    0., ...,  700.,  900.,    0.]], shape=(64, 64))

---

### <span style="color:#96d9f0;">2. Solution Representation</span>

In this section, we define the Wedding Sitting Solution class (`WSSolution`), which extends the base `Solution` class provided in the practical classes. This class represents a candidate solution for the wedding seating optimization problem.

Each solution consists of a list of tables, where each table is a list of guest indices. The class includes methods to generate a valid random initial representation and to evaluate the fitness of a solution based on the total relationship scores between guests seated at the same table.

In [7]:
class WSSolution(Solution):
    def __init__(self, repr=None, relationship_matrix=relationship_matrix, nr_of_tables=8):
        self.relationship_matrix = relationship_matrix
        self.nr_of_tables = nr_of_tables
        self.table_capacity = len(relationship_matrix) // nr_of_tables
        super().__init__(repr=repr)

    def random_initial_representation(self):
        tables = []
        left_idxs = [idx for idx in range(len(self.relationship_matrix))]
        for i in range(self.nr_of_tables):
            tables.append([])
            for j in range(self.table_capacity):
                idx = random.choice(left_idxs)
                left_idxs.remove(idx)
                # Check if idx is already in a table
                #while any(idx in table for table in tables):
                    #idx = random.randint(0, len(self.relationship_matrix) - 1)
                tables[i].append(idx)
        
        return tables

    def fitness(self, verbose=False):
        total_happiness = 0
        i = 1
        for table in self.repr:
            # Sum relationship scores for all unique guest pairs at the table
            table_happiness = sum(self.relationship_matrix[a, b] for a, b in combinations(table, 2))
            total_happiness += table_happiness
            if verbose:
                print(f"Table {i} Happiness = {table_happiness}")
                i += 1
        return total_happiness

In [8]:
solution = WSSolution()

print('Random solution:', solution)
print('Total Fitness:', solution.fitness(verbose=True))

Random solution: [[35, 60, 48, 37, 31, 46, 2, 29], [39, 20, 13, 61, 38, 33, 43, 28], [21, 55, 27, 58, 16, 19, 56, 34], [23, 59, 36, 47, 44, 3, 0, 50], [11, 7, 42, 32, 51, 9, 8, 24], [14, 45, 54, 6, 22, 41, 18, 5], [57, 62, 63, 26, 25, 1, 17, 12], [30, 49, 53, 10, 4, 52, 40, 15]]
Table 1 Happiness = 100.0
Table 2 Happiness = -800.0
Table 3 Happiness = 1300.0
Table 4 Happiness = 600.0
Table 5 Happiness = 1400.0
Table 6 Happiness = 800.0
Table 7 Happiness = 3900.0
Table 8 Happiness = 700.0
Total Fitness: 8000.0


---

### <span style="color:#96d9f0;">3. Selection Algorithms</span>

#### 3.1. Fitness Proportionate Selection

In a typical **fitness proportionate selection** (also known as **roulette wheel selection**), individuals are selected based on their fitness values, with higher fitness values having a greater chance of being chosen. In our optimization problem, however, the fitness values can range from **negative** (indicating conflicts) to **positive** (indicating good relationships between guests).

To adapt the selection process to handle **negative fitness values**, we introduce the following modifications:

1. **Handling Negative Fitness**:
   - Negative fitness values (e.g., -1000) represent undesirable relationships between guests and are valid in our case.
   - However, to ensure that these negative values don't completely block a solution from being selected, we **normalize** the fitness values by adding an **offset** to make all fitness values non-negative.
   - The offset is the **absolute value of the minimum fitness** if it's negative, ensuring that all fitness values become positive or zero without altering the relative fitness ranking of solutions.

2. **Fitness Normalization**:
   - For each individual, we add the **offset** (if necessary) to their fitness value to ensure it's non-negative.
   - This ensures that individuals with negative fitness still have a chance to be selected but are less likely to be chosen compared to those with higher (positive) fitness.

3. **Roulette Wheel Selection**:
   - After normalizing the fitness values, the selection process proceeds as usual, with higher fitness solutions having a higher probability of being selected.
   - By adding the offset, we make sure that **negative fitness values** don't result in zero or near-zero chances of selection, allowing them to contribute to the evolutionary process.

In [9]:
def fitness_proportionate_selection(population: list[Solution], maximization: bool):
    if maximization:
        fitness_values = []
        min_fitness = min(ind.fitness() for ind in population)  # Find the minimum fitness value
        
        # Add a constant offset to ensure no negative fitness values
        offset = abs(min_fitness) if min_fitness < 0 else 0
        
        # Add the offset to each fitness value
        for ind in population:
            fitness_value = ind.fitness() + offset
            fitness_values.append(fitness_value)
    else:
        # Minimization: Use the inverse of the fitness value
        # Lower fitness should have higher probability of being selected
        fitness_values = [1 / ind.fitness() for ind in population]

    total_fitness = sum(fitness_values)
    
    # Generate random number between 0 and total
    random_nr = random.uniform(0, total_fitness)
    
    # For each individual, check if random number is inside the individual's "box"
    box_boundary = 0
    for ind_idx, ind in enumerate(population):
        box_boundary += fitness_values[ind_idx]
        if random_nr <= box_boundary:
            return deepcopy(ind)

In [10]:
population = [WSSolution() for _ in range(10)]

fitness_proportionate_selection_result = fitness_proportionate_selection(population, maximization=True)
print('Fitness Proportionate selection result:', fitness_proportionate_selection_result)

Fitness Proportionate selection result: [[34, 32, 33, 27, 13, 30, 16, 18], [62, 63, 58, 41, 9, 22, 55, 36], [24, 49, 56, 47, 23, 57, 10, 35], [38, 4, 54, 3, 11, 40, 31, 0], [15, 6, 2, 45, 43, 12, 39, 50], [53, 37, 52, 29, 19, 46, 25, 1], [20, 44, 48, 42, 7, 59, 17, 21], [26, 60, 14, 51, 61, 8, 28, 5]]


#### 3.2. Ranking Selection

With Ranking Selection, instead of selecting based on raw fitness values, individuals are ranked by their fitness, and higher-ranked individuals (better seating arrangements) have a higher chance of being selected. This method ensures that negative fitness values aren´t automatically excluded, as selection is based on relative rank rather than absolute fitness. It maintains diversity by giving lower-ranked individuals a chance to contribute, preventing premature convergence.

In [11]:
def ranking_selection(population, maximization=False):
    n_parents = 1  # Always select one parent per call in the GA

    # Sort population based on fitness
    sorted_pop = sorted(
        population,
        key=lambda sol: sol.fitness(),
        reverse=maximization  # reverse=True if maximizing
    )

    # Assign linear ranks
    n = len(sorted_pop)
    ranks = np.arange(1, n + 1)
    total_rank = np.sum(ranks)
    probabilities = ranks / total_rank

    # If maximizing, best is first → reverse probabilities so high rank = high prob
    if maximization:
        probabilities = probabilities[::-1]

    selected = np.random.choice(sorted_pop, size=n_parents, p=probabilities, replace=True)
    return selected[0]  # Return single selected parent


In [12]:
# Create a test population
population = [WSSolution() for _ in range(10)]

# Display fitness of each individual in the population
print("Fitness of individuals in population:")
for i, sol in enumerate(population):
    print(f"Individual {i}: Fitness = {sol.fitness():.2f}")

# Select two individuals using the updated ranking_selection (one at a time)
selected_individuals = [ranking_selection(population, maximization=True) for _ in range(2)]

# Print selected individuals and their fitness
print("\nRanking selection result:")
for i, ind in enumerate(selected_individuals):
    print(f"Selected {i + 1}: {ind}, Fitness = {ind.fitness():.2f}")


Fitness of individuals in population:
Individual 0: Fitness = 9300.00
Individual 1: Fitness = 9200.00
Individual 2: Fitness = 13200.00
Individual 3: Fitness = 13700.00
Individual 4: Fitness = 18700.00
Individual 5: Fitness = 8100.00
Individual 6: Fitness = 12900.00
Individual 7: Fitness = 16600.00
Individual 8: Fitness = 13900.00
Individual 9: Fitness = 9400.00

Ranking selection result:
Selected 1: [[37, 47, 7, 14, 22, 43, 13, 33], [49, 62, 39, 32, 6, 59, 9, 36], [44, 51, 27, 52, 57, 8, 26, 20], [19, 53, 45, 21, 1, 12, 31, 30], [58, 40, 24, 60, 11, 61, 34, 54], [2, 16, 29, 17, 38, 55, 42, 5], [63, 41, 35, 23, 48, 15, 56, 10], [46, 4, 50, 28, 25, 18, 3, 0]], Fitness = 13700.00
Selected 2: [[19, 40, 20, 36, 7, 2, 31, 47], [34, 4, 45, 52, 3, 55, 29, 32], [9, 51, 49, 0, 39, 62, 17, 28], [18, 33, 56, 15, 61, 10, 38, 37], [21, 43, 14, 8, 30, 50, 48, 6], [16, 26, 5, 54, 13, 25, 57, 42], [58, 1, 63, 59, 44, 46, 24, 12], [53, 41, 22, 60, 27, 35, 23, 11]], Fitness = 8100.00


In this case, we can see that the same individual (2nd best in rank) was selected twice. This is okay and makes that it coud happen especially with the high rank and small population.

In [13]:
# Create a test population
population = [WSSolution() for _ in range(10)]

# Display fitness of each individual in the population
print("Fitness of individuals in population:")
for i, sol in enumerate(population):
    print(f"Individual {i}: Fitness = {sol.fitness():.2f}")

# Select two individuals using the updated ranking_selection (one at a time)
selected_individuals = [ranking_selection(population, maximization=True) for _ in range(2)]

# Print selected individuals and their fitness
print("\nRanking selection result:")
for i, ind in enumerate(selected_individuals):
    print(f"Selected {i + 1}: {ind}, Fitness = {ind.fitness():.2f}")

Fitness of individuals in population:
Individual 0: Fitness = 11500.00
Individual 1: Fitness = 16100.00
Individual 2: Fitness = 10800.00
Individual 3: Fitness = 8800.00
Individual 4: Fitness = 15200.00
Individual 5: Fitness = 11600.00
Individual 6: Fitness = 13600.00
Individual 7: Fitness = 13300.00
Individual 8: Fitness = 7800.00
Individual 9: Fitness = 4600.00

Ranking selection result:
Selected 1: [[14, 60, 40, 28, 51, 43, 61, 23], [57, 54, 35, 59, 11, 1, 24, 56], [34, 9, 63, 10, 3, 17, 48, 39], [53, 36, 30, 25, 31, 19, 45, 26], [52, 22, 6, 33, 41, 32, 13, 4], [20, 29, 8, 55, 27, 2, 18, 12], [42, 50, 46, 0, 5, 15, 37, 47], [16, 21, 44, 58, 62, 7, 49, 38]], Fitness = 15200.00
Selected 2: [[40, 13, 1, 15, 51, 4, 20, 5], [16, 57, 37, 18, 29, 10, 41, 38], [28, 26, 19, 2, 60, 39, 56, 45], [63, 62, 21, 54, 43, 3, 35, 53], [55, 46, 36, 33, 50, 0, 7, 14], [6, 58, 52, 9, 30, 17, 59, 44], [27, 25, 8, 11, 22, 47, 49, 31], [61, 42, 32, 12, 48, 24, 23, 34]], Fitness = 8800.00


#### 3.3. Tournament Selection

The tournament selection randomly chooses 𝑘 individuals at random from the population, compares their fitnesses, and returns the best individual out of those to be a parent. 

In [14]:
def tournament_selection(population, k=3):
    competitors = random.sample(population, k)
    return max(competitors, key=lambda ind: ind.fitness())

In [15]:
population = [WSSolution() for _ in range(10)]

tournament_selection_result = tournament_selection(population)
print('Tournament selection result:', tournament_selection_result)

Tournament selection result: [[62, 29, 22, 60, 45, 13, 17, 7], [37, 43, 51, 38, 27, 16, 20, 63], [56, 50, 40, 25, 49, 2, 57, 44], [58, 61, 8, 39, 3, 30, 46, 28], [36, 32, 42, 1, 26, 53, 5, 33], [35, 21, 6, 31, 18, 0, 55, 11], [14, 52, 47, 23, 34, 41, 48, 19], [10, 24, 15, 4, 12, 59, 9, 54]]


To make sure that our tournament_selection implementation truly picks the fittest individual, we wrapped it in a helper function, tournament_verification, which prints out every step of the process. This function is needed because the core algorithm samples competitors internally, not allowing us to check the candidates considered or their fitness except for the winner that is returned. In tournament_verification, we set 
k=10 and explicitly list all ten randomly chosen competitors along with their fitness values, then identify and print the one selected as the fittest by tournament_selection, allowing us to directly compare that result against the ten and confirm it truly has the highest fitness. This gives us clear, reproducible evidence that the tournament selection returns the strongest contender.

In [16]:
def tournament_verification(population, k=10):
    competitors = random.sample(population, k)
    
    print("Fitness of competitors:")
    for i in range(k):
        print(f"Competitor {i+1}: Fitness = {competitors[i].fitness()}")
    
    best_idx, best_competitor = max(enumerate(competitors),key=lambda pair: pair[1].fitness())
    
    print("\nTournament selection result:")
    print(f"Competitor {best_idx+1}: Fitness = {best_competitor.fitness():.2f}")

In [17]:
population = [WSSolution() for _ in range(50)]

tournament_verification(population)

Fitness of competitors:
Competitor 1: Fitness = 7900.0
Competitor 2: Fitness = 12600.0
Competitor 3: Fitness = 13300.0
Competitor 4: Fitness = 12800.0
Competitor 5: Fitness = 14300.0
Competitor 6: Fitness = 13800.0
Competitor 7: Fitness = 13300.0
Competitor 8: Fitness = 14500.0
Competitor 9: Fitness = 9600.0
Competitor 10: Fitness = 15600.0

Tournament selection result:
Competitor 10: Fitness = 15600.00


---

### <span style="color:#96d9f0;">4. Crossover Operators</span>

#### 4.1. Custom Crossover Operator (`table_level_crossover`)

In this section, we implement a custom crossover operator tailored to the constraints of our optimization problem. Unlike standard one-point crossover, which is suitable for flat representations, our solution requires a structured approach:

- Each solution is a nested list representing 8 tables with 8 unique guests each.
- Every guest (from 0 to 63) must appear exactly once across all tables.
- A direct crossover risks creating invalid offspring (with duplicated or missing guests).

To address this, our operator performs table-level crossover:

1. A one-point crossover is applied between the two parent solutions at the table level. This can result in invalid intermediate offspring that contain duplicate guests and omit others, violating the uniqueness constraint.

2. The resulting child is then repaired by:

    - Detecting duplicate and missing guests
    - Replacing duplicates with missing ones to restore a valid configuration
    - This ensures all offspring are valid solutions and suitable for use in the genetic algorithm.

In [18]:
def table_level_crossover(parent1, parent2):
    # Deep copies of the parent representations
    p1_repr = deepcopy(parent1.repr)
    p2_repr = deepcopy(parent2.repr)

    # Flatten guest lists
    total_guests = len(parent1.relationship_matrix)

    # One-point crossover at table level
    xo_point = random.randint(1, len(p1_repr) - 1)
    child1_repr = p1_repr[:xo_point] + p2_repr[xo_point:]
    child2_repr = p2_repr[:xo_point] + p1_repr[xo_point:]

    def repair(child_repr):
        # Count how many times each guest appears
        guest_counts = {}
        for table in child_repr:
            for guest in table:
                guest_counts[guest] = guest_counts.get(guest, 0) + 1

        # Identify missing and extra guests
        missing = [g for g in range(total_guests) if guest_counts.get(g, 0) == 0]
        extras = [g for g, count in guest_counts.items() if count > 1 for _ in range(count - 1)]

        # Replace extra guests with missing ones
        for table in child_repr:
            for i in range(len(table)):
                guest = table[i]
                if guest_counts[guest] > 1:
                    new_guest = missing.pop()
                    guest_counts[guest] -= 1
                    table[i] = new_guest
                    guest_counts[new_guest] = 1  # added to the table

        return child_repr

    # Repair both children
    repaired1 = repair(child1_repr)
    repaired2 = repair(child2_repr)

    return WSSolution(repr=repaired1), WSSolution(repr=repaired2)

In [19]:
# Create 2 solutions
parent1 = WSSolution()
parent2 = WSSolution()

# Parents before crossover
print("Parent 1 Seating Arrangement:")
print(parent1.repr)
print("Parent 2 Seating Arrangement:")
print(parent2.repr)

# Apply PMX crossover
offspring1, offspring2 = table_level_crossover(parent1, parent2)

# Print offspring after crossover
print("\nOffspring 1 Seating Arrangement:")
print(offspring1.repr)
print("Offspring 2 Seating Arrangement:")
print(offspring2.repr)

# Compare the fitness values to see how they change
print("\nParent 1 Fitness:", parent1.fitness())
print("Parent 2 Fitness:", parent2.fitness())
print("Offspring 1 Fitness:", offspring1.fitness())
print("Offspring 2 Fitness:", offspring2.fitness())


Parent 1 Seating Arrangement:
[[18, 1, 27, 23, 62, 63, 0, 8], [53, 52, 49, 11, 44, 14, 55, 9], [57, 22, 56, 19, 17, 43, 16, 35], [26, 48, 60, 61, 21, 30, 38, 45], [6, 13, 33, 24, 39, 59, 28, 3], [54, 12, 42, 29, 37, 2, 40, 50], [36, 10, 20, 5, 4, 58, 7, 47], [51, 25, 15, 31, 34, 46, 41, 32]]
Parent 2 Seating Arrangement:
[[5, 10, 44, 9, 27, 21, 60, 8], [12, 43, 18, 0, 37, 16, 59, 30], [38, 42, 35, 62, 13, 17, 55, 49], [26, 1, 50, 56, 3, 4, 52, 54], [57, 45, 20, 23, 46, 40, 28, 11], [31, 39, 51, 47, 41, 19, 22, 58], [61, 48, 33, 7, 6, 63, 53, 36], [15, 34, 25, 2, 14, 29, 24, 32]]

Offspring 1 Seating Arrangement:
[[18, 60, 27, 59, 43, 37, 0, 8], [30, 21, 16, 12, 44, 10, 5, 9], [38, 42, 35, 62, 13, 17, 55, 49], [26, 1, 50, 56, 3, 4, 52, 54], [57, 45, 20, 23, 46, 40, 28, 11], [31, 39, 51, 47, 41, 19, 22, 58], [61, 48, 33, 7, 6, 63, 53, 36], [15, 34, 25, 2, 14, 29, 24, 32]]
Offspring 2 Seating Arrangement:
[[63, 62, 44, 9, 27, 55, 53, 8], [52, 49, 18, 0, 23, 14, 11, 1], [57, 22, 56, 19, 17

In [20]:
def check_no_repeated_guests(individual):
    seen_guests = []  # Keep track of guests we've seen
    for table in individual:
        for guest in table:
            if guest in seen_guests:  # If guest is already in the list, they are repeated
                print(f"Repeated guest found: {guest}")  # Print the repeated guest for debugging
                return False
            seen_guests.append(guest)  # Add guest to the list
    return True  # No repeated guests found

# Check if there are repeated guests in the offspring seating arrangements
print("Offspring 1 has no repeated guests:", check_no_repeated_guests(offspring1.repr))
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(offspring2.repr))

Offspring 1 has no repeated guests: True
Offspring 2 has no repeated guests: True


#### 4.2. Partially Matched Crossover

The Partially Matched Crossover (pmc) is a genetic algorithm operator designed for permutations, like our guest seating arrangement. It swaps a segment between two parent solutions and builds offspring by preserving the order and position of elements while avoiding duplicates. In our implementation, we adapted PMX to work with the wedding seating format by flattening table arrangements into 1D lists, applying PMX, and then converting them back. We carefully resolved conflicts during crossover by mapping duplicate entries to valid unused guests, ensuring that each guest appears exactly once in the final seating—maintaining valid, non-repetitive arrangements.

In [21]:
def partially_matched_crossover(parent1, parent2):
    """
    PMC crossover for WSSolution-based seating plans with 8 tables.
    Ensures no repeated guests in offspring.
    """
    import random

    def flatten(tables):
        return [guest for table in tables for guest in table]

    def unflatten(guest_list, table_size):
        return [guest_list[i:i + table_size] for i in range(0, len(guest_list), table_size)]

    parent1_flat = flatten(parent1.repr)
    parent2_flat = flatten(parent2.repr)
    size = len(parent1_flat)
    table_size = parent1.table_capacity

    # Choose two crossover points
    point1, point2 = sorted(random.sample(range(size), 2))

    def pmc(p1, p2):
        child = [None] * size
        # Copy the crossover segment from p1
        child[point1:point2] = p1[point1:point2]

        for i in range(point1, point2):
            if p2[i] not in child:
                val = p2[i]
                idx = i
                while True:
                    mapped = p1[idx]
                    idx = p2.index(mapped)
                    if child[idx] is None:
                        child[idx] = val
                        break

        # Fill remaining positions from p2
        for i in range(size):
            if child[i] is None:
                child[i] = p2[i]

        return child

    child1_flat = pmc(parent1_flat, parent2_flat)
    child2_flat = pmc(parent2_flat, parent1_flat)

    # Convert back to table format
    child1_repr = unflatten(child1_flat, table_size)
    child2_repr = unflatten(child2_flat, table_size)

    # Wrap in WSSolution
    offspring1 = WSSolution(repr=child1_repr, nr_of_tables=parent1.nr_of_tables)
    offspring2 = WSSolution(repr=child2_repr, nr_of_tables=parent2.nr_of_tables)

    return offspring1, offspring2


In [22]:
# Create 2 solutions
parent1 = WSSolution()
parent2 = WSSolution()

# Parents before crossover
print("Parent 1 Seating Arrangement:")
print(parent1.repr)
print("Parent 2 Seating Arrangement:")
print(parent2.repr)

# Apply PMX crossover
offspring1, offspring2 = partially_matched_crossover(parent1, parent2)

# Print offspring after crossover
print("\nOffspring 1 Seating Arrangement:")
print(offspring1.repr)
print("Offspring 2 Seating Arrangement:")
print(offspring2.repr)

# Compare the fitness values to see how they change
print("\nParent 1 Fitness:", parent1.fitness())
print("Parent 2 Fitness:", parent2.fitness())
print("Offspring 1 Fitness:", offspring1.fitness())
print("Offspring 2 Fitness:", offspring2.fitness())


Parent 1 Seating Arrangement:
[[39, 23, 34, 62, 4, 30, 29, 59], [51, 11, 9, 10, 53, 31, 56, 48], [24, 43, 50, 20, 60, 36, 15, 44], [2, 52, 58, 25, 41, 49, 8, 18], [54, 40, 57, 42, 61, 19, 17, 27], [46, 3, 45, 21, 12, 55, 38, 37], [32, 5, 1, 28, 14, 13, 63, 22], [6, 33, 7, 0, 47, 26, 35, 16]]
Parent 2 Seating Arrangement:
[[37, 39, 61, 10, 21, 45, 43, 26], [57, 15, 14, 22, 59, 55, 16, 4], [29, 13, 2, 28, 56, 58, 33, 62], [11, 49, 60, 42, 46, 7, 17, 35], [18, 9, 63, 40, 0, 30, 23, 47], [41, 19, 32, 31, 34, 36, 50, 27], [20, 25, 38, 54, 1, 12, 51, 3], [53, 6, 5, 8, 52, 44, 24, 48]]

Offspring 1 Seating Arrangement:
[[9, 39, 23, 10, 31, 20, 43, 44], [51, 15, 50, 30, 59, 36, 16, 4], [29, 34, 11, 35, 56, 60, 53, 62], [2, 52, 58, 25, 41, 49, 8, 18], [54, 40, 57, 42, 61, 19, 17, 27], [46, 3, 45, 21, 12, 55, 38, 37], [32, 5, 1, 28, 14, 13, 63, 22], [6, 33, 7, 0, 47, 26, 24, 48]]
Offspring 2 Seating Arrangement:
[[39, 61, 13, 62, 4, 22, 29, 59], [57, 2, 37, 10, 33, 21, 56, 48], [24, 43, 14, 45, 

In [23]:
# Check if there are repeated guests in the offspring seating arrangements
print("Offspring 1 has no repeated guests:", check_no_repeated_guests(offspring1.repr))
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(offspring2.repr))

Offspring 1 has no repeated guests: True
Offspring 2 has no repeated guests: True


#### 4.3. Order Crossover

The Order Crossover (OX) is a genetic algorithm operator also designed for permutation problems, such as our guest seating arrangements. It selects a contiguous block of seats from one parent and preserves both the relative order and positional integrity of the remaining guests by filling in, in sequence, from the other parent’s list and skipping any guest already included. In our implementation, we adapt OX to the wedding format by flattening each table layout into a single list, choosing two crossover points, copying the segment from Parent 1 into Offspring 1 (and vice versa for Offspring 2), then rotating through the other parent’s flattened list to fill the empty slots while avoiding duplicates. Finally, we reshape the 1D offspring lists back into the original table structure, guaranteeing a complete, non-repetitive seating plan.

In [24]:
def order_crossover(parent1, parent2):
    """
    Order crossover for WSSolution-based seating plans with 8 tables.
    Ensures no repeated guests in offspring.
    """
    def flatten(tables):
        return [guest for table in tables for guest in table]

    def unflatten(guest_list, table_size):
        return [guest_list[i:i + table_size] for i in range(0, len(guest_list), table_size)]

    parent1_flat = flatten(parent1.repr)
    parent2_flat = flatten(parent2.repr)
    size = len(parent1_flat)
    table_size = parent1.table_capacity

    # Choose two crossover points
    point1, point2 = sorted(random.sample(range(size), 2))

    def order(p1, p2):
        child = [None] * size
        child[point1:point2] = p1[point1:point2]
        
        for i in range(point2-size, point1):
            val = p2[i]
            idx = i
            while val in child:
                idx+=1
                val = p2[idx]
            child[i] = val

        return child

    child1_flat = order(parent1_flat, parent2_flat)
    child2_flat = order(parent2_flat, parent1_flat)

    # Convert back to table format
    child1_repr = unflatten(child1_flat, table_size)
    child2_repr = unflatten(child2_flat, table_size)

    # Wrap in WSSolution
    offspring1 = WSSolution(repr=child1_repr, nr_of_tables=parent1.nr_of_tables)
    offspring2 = WSSolution(repr=child2_repr, nr_of_tables=parent2.nr_of_tables)

    return offspring1, offspring2

In [25]:
# Create 2 solutions
parent1 = WSSolution()
parent2 = WSSolution()

# Parents before crossover
print("Parent 1 Seating Arrangement:")
print(parent1.repr)
print("Parent 2 Seating Arrangement:")
print(parent2.repr)

# Apply Order crossover
offspring1, offspring2 = order_crossover(parent1, parent2)

# Print offspring after crossover
print("\nOffspring 1 Seating Arrangement:")
print(offspring1.repr)
print("Offspring 2 Seating Arrangement:")
print(offspring2.repr)

# Compare the fitness values to see how they change
print("\nParent 1 Fitness:", parent1.fitness())
print("Parent 2 Fitness:", parent2.fitness())
print("Offspring 1 Fitness:", offspring1.fitness())
print("Offspring 2 Fitness:", offspring2.fitness())

Parent 1 Seating Arrangement:
[[55, 17, 26, 21, 10, 44, 15, 54], [32, 7, 57, 11, 31, 6, 49, 22], [60, 40, 33, 19, 25, 34, 12, 28], [16, 2, 56, 61, 51, 13, 52, 14], [42, 63, 24, 53, 23, 3, 47, 45], [35, 41, 39, 8, 1, 48, 37, 18], [29, 36, 58, 59, 30, 5, 62, 46], [0, 43, 20, 4, 50, 27, 38, 9]]
Parent 2 Seating Arrangement:
[[16, 62, 36, 21, 10, 15, 13, 25], [6, 44, 0, 34, 54, 35, 47, 19], [53, 37, 33, 31, 56, 45, 18, 51], [9, 46, 52, 5, 2, 12, 1, 28], [58, 29, 48, 3, 50, 24, 30, 40], [59, 39, 49, 7, 20, 41, 17, 23], [60, 11, 14, 38, 57, 26, 63, 61], [22, 8, 4, 55, 43, 32, 27, 42]]

Offspring 1 Seating Arrangement:
[[36, 21, 10, 15, 13, 25, 6, 44], [0, 34, 54, 19, 33, 31, 56, 18], [51, 9, 46, 52, 5, 2, 12, 28], [58, 29, 50, 30, 40, 59, 49, 7], [20, 17, 24, 53, 23, 3, 47, 45], [35, 41, 39, 8, 1, 48, 37, 60], [11, 14, 38, 57, 26, 63, 61, 22], [4, 55, 43, 32, 27, 42, 16, 62]]
Offspring 2 Seating Arrangement:
[[44, 15, 54, 32, 57, 11, 31, 6], [22, 60, 33, 19, 25, 34, 12, 28], [16, 2, 56, 61, 

In [26]:
# Check if there are repeated guests in the offspring seating arrangements
print("Offspring 1 has no repeated guests:", check_no_repeated_guests(offspring1.repr))
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(offspring2.repr))

Offspring 1 has no repeated guests: True
Offspring 2 has no repeated guests: True


---

### <span style="color:#96d9f0;">5. Mutation Operators</span>

#### 5.1. Swap Mutation

The swap mutation operator introduces random changes to the seating arrangement by swapping two guests between different tables.

It's important to note that swapping two guests within the same table would not create any new configuration - it would simply result in the same solution as before. Since we want to introduce meaningful changes, the swap mutation ensures that the two guests are always swapped between different tables. This prevents the mutation from generating an invalid or identical solution.

This mutation helps the genetic algorithm explore new seating arrangements by shuffling guests across tables while respecting the overall constraints of the problem.

In [27]:
def swap_mutation(solution, mut_prob):
    if random.random() >= mut_prob:
        return solution  # No mutation; return original

    # Create a deep copy of the solution
    new_solution = deepcopy(solution)

    tables = new_solution.repr
    if len(tables) < 2:
        return new_solution

    # Pick two random tables
    t1, t2 = random.sample(range(len(tables)), 2)
    if not tables[t1] or not tables[t2]:
        return new_solution

    # Swap one person from each
    i1 = random.randint(0, len(tables[t1]) - 1)
    i2 = random.randint(0, len(tables[t2]) - 1)
    tables[t1][i1], tables[t2][i2] = tables[t2][i2], tables[t1][i1]

    return new_solution

In [28]:
solution = WSSolution()

mutated_solution = swap_mutation(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

Original solution: [[15, 55, 1, 9, 8, 31, 29, 58], [46, 11, 14, 33, 19, 27, 39, 20], [10, 2, 23, 60, 35, 45, 16, 52], [61, 54, 37, 21, 22, 40, 3, 57], [43, 4, 0, 42, 41, 38, 30, 18], [62, 44, 53, 36, 28, 13, 51, 56], [49, 26, 17, 5, 12, 32, 47, 59], [24, 7, 25, 48, 6, 50, 34, 63]]
Mutated solution: [[15, 55, 1, 26, 8, 31, 29, 58], [46, 11, 14, 33, 19, 27, 39, 20], [10, 2, 23, 60, 35, 45, 16, 52], [61, 54, 37, 21, 22, 40, 3, 57], [43, 4, 0, 42, 41, 38, 30, 18], [62, 44, 53, 36, 28, 13, 51, 56], [49, 9, 17, 5, 12, 32, 47, 59], [24, 7, 25, 48, 6, 50, 34, 63]]


#### 5.2. Inversion Mutation

The inversion mutation operator introduces variation by selecting a random segment of the seating plan and reversing the order of guests within that segment.

Since the seating arrangement is represented as a list of tables, the entire structure is first flattened into a single list containing all guests. The inversion is then performed on a random sublist, effectively inverting a continuous sequence of guests. After the inversion, the list is reshaped back into the original table format.

This mutation allows the algorithm to explore new configurations by reordering guests while preserving the structure of the solution. To ensure validity, a repair function is applied after mutation to guarantee that all guests remain uniquely assigned with no duplicates or omissions.

In [29]:
def inversion_mutation_seating(individual: WSSolution, mut_prob):
    # Access the seating arrangement (8 tables with guests)
    tables = individual.repr

    rows = len(tables)
    cols = len(tables[0])
    
    # Flatten the seating arrangement into a single list of guests
    flat = [guest for table in tables for guest in table]

    # Apply inversion mutation with the given probability
    if random.random() < mut_prob:
        # Select two random indices for the inversion
        start_row, end_row = sorted(random.sample(range(rows), 2))
        start_col, end_col = random.sample(range(cols), 2)
        start = start_row * cols + start_col
        end = end_row * cols + end_col
        # Reverse the segment between the two indices
        flat[start:end + 1] = reversed(flat[start:end + 1])

    # Reshape the flat list back into 8 tables of 8 guests each
    new_tables = [flat[i:i + individual.table_capacity] for i in range(0, len(flat), individual.table_capacity)]

    # Return a new WSSolution with the mutated seating arrangement
    return WSSolution(repr=new_tables, relationship_matrix=individual.relationship_matrix, nr_of_tables=individual.nr_of_tables)

In [30]:
solution = WSSolution()

mutated_solution = inversion_mutation_seating(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

Original solution: [[47, 31, 33, 1, 5, 59, 28, 23], [37, 50, 51, 44, 21, 26, 52, 19], [12, 9, 36, 8, 60, 46, 41, 29], [49, 54, 0, 58, 63, 2, 61, 30], [56, 22, 39, 6, 40, 11, 20, 57], [4, 14, 15, 25, 42, 34, 48, 13], [27, 18, 53, 24, 17, 7, 3, 32], [38, 62, 43, 35, 45, 10, 55, 16]]
Mutated solution: [[47, 31, 33, 1, 5, 59, 57, 20], [11, 40, 6, 39, 22, 56, 30, 61], [2, 63, 58, 0, 54, 49, 29, 41], [46, 60, 8, 36, 9, 12, 19, 52], [26, 21, 44, 51, 50, 37, 23, 28], [4, 14, 15, 25, 42, 34, 48, 13], [27, 18, 53, 24, 17, 7, 3, 32], [38, 62, 43, 35, 45, 10, 55, 16]]


#### 5.3. Scramble Mutation

The scramble mutation operator introduces variation by selecting a random subset of guests in the seating plan and scrambling their order within the selected segment. Unlike the inversion mutation, where the order is reversed, the scramble mutation randomizes the order of guests in a selected sublist.

The process begins by flattening the seating arrangement, which is represented as a list of tables, into a single list of guests. Then, a random segment of guests is selected, and the order of the guests within that segment is scrambled. After the scrambling is performed, the list is reshaped back into the original table structure.

This mutation operator allows the algorithm to explore different configurations by rearranging guests within a specified segment of the seating arrangement, offering a broader search of potential solutions. It maintains the structure of the solution by preserving the overall number of guests per table while allowing the order of guests to vary.

To ensure validity and correctness of the solution, a repair function is applied after the mutation to guarantee that all guests remain uniquely assigned, with no duplicates or omissions.

In [31]:
def scramble_mutation(solution, mut_prob):
    if random.random() >= mut_prob:
        return solution  # No mutation; return original

    # Create a deep copy of the solution
    new_solution = deepcopy(solution)

    tables = new_solution.repr
    rows = len(tables)
    cols = len(tables[0])

    if rows < 2:
        return new_solution

    selected_cols = [random.randint(0, cols-1) for _ in range(rows)]
    selected_positions = [(row, selected_cols[row]) for row in range(rows)]

    selected_values = []
    for row, col in selected_positions: 
        if not tables[row][col]:
            return new_solution
        else: 
            selected_values.append(tables[row][col])
    
    def deranged_shuffle(lst):
        while True:
            shuffled = lst[:] 
            random.shuffle(shuffled)
            if all(a != b for a, b in zip(lst, shuffled)):
                return shuffled
    
    selected_values = deranged_shuffle(selected_values)

    for (row, col), value in zip(selected_positions, selected_values):
        tables[row][col] = value

    return new_solution

In [32]:
solution = WSSolution()

mutated_solution = scramble_mutation(solution, 1) # Meti a probabilidade de mutação = 1 para garantir que a mutação acontece e ver se funciona
print('Original solution:', solution)
print('Mutated solution:', mutated_solution)

Original solution: [[18, 8, 1, 39, 60, 45, 26, 42], [63, 28, 46, 19, 48, 35, 20, 33], [61, 30, 53, 58, 23, 13, 12, 54], [2, 40, 21, 9, 17, 31, 5, 50], [36, 62, 11, 15, 16, 52, 43, 29], [37, 24, 22, 59, 25, 44, 3, 14], [57, 7, 41, 4, 34, 49, 6, 0], [10, 55, 38, 56, 47, 32, 27, 51]]
Mutated solution: [[15, 8, 1, 39, 60, 45, 26, 42], [63, 22, 46, 19, 48, 35, 20, 33], [61, 7, 53, 58, 23, 13, 12, 54], [2, 40, 21, 9, 17, 31, 56, 50], [36, 62, 11, 5, 16, 52, 43, 29], [37, 24, 18, 59, 25, 44, 3, 14], [57, 30, 41, 4, 34, 49, 6, 0], [10, 55, 38, 28, 47, 32, 27, 51]]


### <span style="color:#96d9f0;">6. Random Trials</span>

import time

start = time.time()
for i in range(10):
    res = 0
    scores = []
    for j in range(100000):
        solution = WSSolution()
        res=solution.fitness()
        scores.append(res)
    avg_score = sum(scores)/100000
    best_score = max(scores)
    print(f"Result {i+1}: Average Fitness: {avg_score} and Best Fitness Found {best_score}")

end = time.time()

print(f"Time elapsed: {int((end- start)//60)} minutes and {round((end-start)%60)} seconds")

If 1000000 solutions took around 2 minutes, then the total seating arrangements (assuming a much simpler scenario where all random solutions displayed never repeat) would be:

import math

total_combinations = (math.comb(64,8)*math.comb(56,8)*math.comb(48, 8)*math.comb(40, 8)*math.comb(32, 8)*math.comb(24, 8)*math.comb(16,8)) / math.factorial(8)
minutes_taken = total_combinations * 2 / 1000000
years_taken = minutes_taken/(60*24*365) 


print(f" There are {total_combinations} different seating arrangements and it would take around {years_taken} years for a computer to compare all the solutions and find the best.")

---

### <span style="color:#96d9f0;">7. Genetic Algorithm</span>

In [33]:
def genetic_algorithm(
    initial_population: list[Solution],
    max_gen: int,
    selection_algorithm: Callable,
    crossover_operator: Callable,
    mutation_operator: Callable,
    maximization: bool = False,
    xo_prob: float = 0.9,
    mut_prob: float = 0.2,
    elitism: bool = True,
    verbose: bool = False,
):
    """
    Executes a genetic algorithm to optimize a population of solutions.

    Args:
        initial_population (list[Solution]): The starting population of solutions.
        max_gen (int): The maximum number of generations to evolve.
        selection_algorithm (Callable): Function used for selecting individuals.
        crossover_operator (Callable): Function to perform crossover between two individuals.
        mutation_operator (Callable): Function to mutate an individual.
        maximization (bool, optional): If True, maximizes the fitness function; otherwise, minimizes. Defaults to False.
        xo_prob (float, optional): Probability of applying crossover. Defaults to 0.9.
        mut_prob (float, optional): Probability of applying mutation. Defaults to 0.2.
        elitism (bool, optional): If True, carries the best individual to the next generation. Defaults to True.
        verbose (bool, optional): If True, prints detailed logs for debugging. Defaults to False.

    Returns:
        Solution: The best solution found after evolving for max_gen generations.
    """
    population = initial_population

    for gen in range(1, max_gen + 1):
        if verbose:
            print(f'-------------- Generation: {gen} --------------')

        new_population = []

        if elitism:
            new_population.append(deepcopy(get_best_ind(population, maximization)))

        while len(new_population) < len(population):
            parent1 = selection_algorithm(population, maximization)
            parent2 = selection_algorithm(population, maximization)

            if verbose:
                print(f'Selected parents:\n{parent1}\n{parent2}')

            if random.random() < xo_prob:
                offspring1, offspring2 = crossover_operator(parent1, parent2)
                if verbose:
                    print('Crossover applied.')
            else:
                offspring1, offspring2 = deepcopy(parent1), deepcopy(parent2)
                if verbose:
                    print('Replication applied.')

            mutated1 = mutation_operator(offspring1, mut_prob)
            new_population.append(mutated1)
            if verbose:
                print(f'Mutated offspring 1: {mutated1}')

            if len(new_population) < len(population):
                mutated2 = mutation_operator(offspring2, mut_prob)
                new_population.append(mutated2)
                if verbose:
                    print(f'Mutated offspring 2: {mutated2}')

        population = new_population

        if verbose:
            best = get_best_ind(population, maximization)
            print(f'Best individual fitness this generation: {best.fitness()}')

    return get_best_ind(population, maximization)

In [ ]:
# Grid search
# Define parameter ranges
selection_methods = [fitness_proportionate_selection, ranking_selection, tournament_selection]
crossover_methods = [table_level_crossover, partially_matched_crossover, order_crossover]
mutation_methods = [swap_mutation, inversion_mutation_seating, scramble_mutation]
mutation_probs = [random.uniform(0, 1/3), random.uniform(1/3, 2/3), random.uniform(2/3, 1)]
crossover_probs = [random.uniform(0, 1/3), random.uniform(1/3, 2/3), random.uniform(2/3, 1)]
elitism = [True, False]

max_gen = 100

population_size = 50
population = [WSSolution() for _ in range(population_size)]

# List to store the top results
top_results = [] 
# Calculate total number of combinations for the progress bar
total_combinations = len(selection_methods) * len(crossover_methods) * len(mutation_methods) * len(mutation_probs) * len(crossover_probs) * len(elitism)

for selection, crossover, mutation, mut_prob, xo_prob, elitism in tqdm(
    product(selection_methods, crossover_methods, mutation_methods, mutation_probs, crossover_probs, elitism),
    total=total_combinations,
    desc="Grid Search Progress"
):
    best_solution = genetic_algorithm(
        initial_population=population,
        max_gen=max_gen,
        selection_algorithm=selection,
        crossover_operator=crossover,
        mutation_operator=mutation,
        maximization=True,
        xo_prob=xo_prob,
        mut_prob=mut_prob,
        elitism=elitism,
        verbose=False
    )
    fitness = best_solution.fitness()
    params = (selection.__name__, crossover.__name__, mutation.__name__, mut_prob, xo_prob, elitism)

    # Append the results to top_results and sort the list by fitness to keep only the top 5 results
    top_results.append((fitness, params))
    top_results = sorted(top_results, key=lambda x: x[0], reverse=True)[:5]

print("Top 5 Parameter Combinations:")
for rank, (fitness, params) in enumerate(top_results, start=1):
    print(f"#{rank}: Fitness = {fitness}, Parameters = {params}")

Grid Search Progress: 100%|██████████| 486/486 [2:42:03<00:00, 20.01s/it]  

Top 5 Parameter Combinations:
#1: Fitness = 70000.0, Parameters = ('ranking_selection', 'table_level_crossover', 'swap_mutation', 0.30510595081712133, 0.7242350443893829, False)
#2: Fitness = 69300.0, Parameters = ('ranking_selection', 'table_level_crossover', 'swap_mutation', 0.5134909741659257, 0.4152874637630008, True)
#3: Fitness = 69300.0, Parameters = ('ranking_selection', 'partially_matched_crossover', 'swap_mutation', 0.5134909741659257, 0.7242350443893829, True)
#4: Fitness = 68300.0, Parameters = ('ranking_selection', 'table_level_crossover', 'swap_mutation', 0.30510595081712133, 0.7242350443893829, True)
#5: Fitness = 68300.0, Parameters = ('ranking_selection', 'table_level_crossover', 'inversion_mutation_seating', 0.5134909741659257, 0.4152874637630008, True)


In [ ]:
# Number of individuals in the initial population
population_size = 50

# Generate initial population of random WSSolutions
population = [WSSolution() for _ in range(population_size)]

In [ ]:
best = genetic_algorithm(
    initial_population=population,
    max_gen=2000,
    selection_algorithm=ranking_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    mut_prob=0.3,
    xo_prob=0.8,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(best)
print("Fitness:", best.fitness())

In [ ]:
isa_try = genetic_algorithm(
    initial_population=population,
    max_gen=2000,
    selection_algorithm=ranking_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(isa_try)
print("Fitness:", isa_try.fitness())

In [ ]:
#Number of individuals in the initial population
population_size2 = 3

# Generate initial population of random WSSolutions
population2 = [WSSolution() for _ in range(population_size2)]

best2 = genetic_algorithm(
    initial_population=population2,
    max_gen=5,
    selection_algorithm=tournament_selection,
    crossover_operator=table_level_crossover,
    mutation_operator=inversion_mutation_seating,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution found:")
print(best2)
print("Fitness:", best2.fitness())

In [ ]:
# Define initial population
population = [WSSolution(nr_of_tables=8) for _ in range(10)]

# Run the genetic algorithm with PMX for just 2 generations
pmc_test = genetic_algorithm(
    initial_population=population,
    max_gen=2,
    selection_algorithm=fitness_proportionate_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

# Output best result
print("\nBest solution after 2 generations:")
print(pmc_test.repr)
print("Fitness:", pmc_test.fitness())

In [ ]:
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(pmc_test.repr))

In [ ]:
# Define initial population
population = [WSSolution(nr_of_tables=8) for _ in range(10)]

# Run the genetic algorithm with PMX for just 2 generations
ranking_test = genetic_algorithm(
    initial_population=population,
    max_gen=2,
    selection_algorithm=ranking_selection,
    crossover_operator=partially_matched_crossover,
    mutation_operator=swap_mutation,
    maximization=True,
    xo_prob=0.9,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

# Output best result
print("\nBest solution after 2 generations:")
print(ranking_test.repr)
print("Fitness:", ranking_test.fitness())

In [ ]:
print("Offspring 2 has no repeated guests:", check_no_repeated_guests(ranking_test.repr))